# Final Project: Phase 1

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import json

## Loading our Dataset

Let's say we're an up and coming YouTuber (circa 2011-2014), and our goal is to get as popular as fast as possible. To do this, we ask: what types of videos trend and do the best on YouTube?

In [ ]:
df = pd.read_csv("./project_data/USVideos.csv")

In [ ]:
df.shape

In [ ]:
df.head()

We have to connect our data from our .json file an meld it into our table.

In [ ]:
with open("./project_data/US_category_id.json") as f:
    categories = json.load(f)

cat_map = {}
for index,cat in enumerate(categories["items"]):
    cat_map[int(cat["id"])]=cat["snippet"]["title"]

In [ ]:
df["category"] = df["category_id"].map(cat_map)

In [ ]:
df.head()

## Exploratory Data Analysis

First, it's always good to get a 5 number summary of our data.

In [ ]:
df.describe()

Of course, the numbers for `catagory_id` mean nothing since it's a catagorical attribute.

Let's look at it for real

In [ ]:
df["category"].describe()

In [ ]:
countplot = sns.countplot(x=df["category"])
countplot.set_xticklabels(countplot.get_xticklabels(),rotation=90)

### Selecting a Metric

Now, what do we mean by "do best"? We have four possible numeric items that could signal quality, `views`,`likes`,`dislikes`, and `comment_count`. Do we want to optimize for all of them, or just one?

Let's dig into these variables a bit and figure out what to expect.

In [ ]:
# Making a subset of the dataframe
numbers = df[["views","likes","dislikes","comment_count"]]

In [ ]:
numbers

In [ ]:
# We sample a small amount because trying to plot all 40000 rows would be way too slow.
small_sample = numbers.sample(n=100)

In [ ]:
sns.pairplot(small_sample)

Hm, looking at our data, it seems like a lot of it is clustered before growing exponentially. That means just running a simple linear correlation would be unwise.

Also the image is too small, so let's dig a bit more into one variable. 
Let's take `views` as an example.

In [ ]:
sns.histplot(df["views"])

Hm, not much more info. What if we plot it on a log scale?

In [ ]:
sns.histplot(df["views"],log_scale=True)

Woah! That looks *a lot* like a normal distribution. In fact, this is what we call a [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution). While this isn't something we'll go in depth, know that normally distributed data plays very nice with many statistical methods: so it's always a good sign.

Using this, let's do a pair plot again, except with all of our data log transformed.

Since we're doing a log transform, we'll run into problems with taking the log of zero values (because $log(0)=-infinity$). 

One of the most common workarounds for this is to just add 1 to each variable, then take the log.

In [ ]:
log_numbers = np.log(numbers+1)

In [ ]:
sns.pairplot(log_numbers)

Now that's looking pretty correlated to me. (Aside from the row of points along each axis)

Now we can see if there's any redundant, highly correlated features.

In [ ]:
log_numbers.corr(method="pearson")

Looks like `(likes, views)` and `(comment count, likes)` are highly correlated. So if we optimize views, likes will probably follow.

**Thus, our goal is to try and predict how many views a video will get based off of the title, description, and tags.**

### Dealing with Dates and Time

Now, for our model, we're not going to use the publish or trending dates. However, it's useful to see how to deal with dates and time when processing data.

Another interesting question. When do videos get popular compared to their publish date?

In [ ]:
times = df[["trending_date","publish_time"]]

In [ ]:
times

These dates are currently in string format. If we want to do more operations on them, we should transform them into a more rich datetime format.

In [ ]:
trending_times = pd.to_datetime(times["trending_date"],format="%y.%d.%m")
trending_times

In [ ]:
# Now, this doesn't look that much different, but now we can do a bunch more interesting conversions. For example, let's get the time in seconds:
# Converting dates into seconds make them easier to plot
trending_times.values.astype(float)

We need to massage the publish time a bit more. Since we don't care about hours and seconds, we can just lop everything past the `T` seperator off.

In [ ]:
publish_times = times["publish_time"].str.slice(0,-14)
publish_times.head()

In [ ]:
publish_times = pd.to_datetime(publish_times,format="%Y.%m.%d")
publish_times

In [ ]:
new_times = pd.DataFrame(data={"publish_times":publish_times,"trending_times":trending_times})

In [ ]:
new_times["difference"] = new_times["trending_times"]-new_times["publish_times"]
new_times

In [ ]:
# Let's get a 5 number summary of the difference
new_times.describe()

## Preprocessing

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Since our data is a list of tags: we have to be a bit more clever then just doing bag of words on the whole thing. Since for example, the tag "nicholas cage" is different from the tags "nicholas" and "cage".

We use some regex so that our processor captures all of the phrases seperated by a pipe as one word. In addition to this, we strip out all the extra quotes.

In [ ]:
df["tags"].iloc[10]

In [ ]:
vectorizer = CountVectorizer(token_pattern=r"([^|]+)")
vectorizer.fit_transform([df["tags"].iloc[10].replace("\"","")])
vectorizer.get_feature_names_out()

Now let's vectorize the whole dataset

In [ ]:
tag_vectorizer = CountVectorizer(token_pattern=r"([^|]+)")
tag_vectors = tag_vectorizer.fit_transform(df["tags"].str.replace("\"",""))

In [ ]:
tag_vectors.shape

We can do our simple vectorization that we've done before for the descriptions.

In [ ]:
df["description"]

In [ ]:
df["description"]

In [ ]:
description_vectorizer = CountVectorizer()
description_vectors = description_vectorizer.fit_transform(df["description"].values.astype('U'))

In [ ]:
description_vectors.shape

In [ ]:
title_vectorizer = CountVectorizer()
title_vectors = title_vectorizer.fit_transform(df["title"])

In [ ]:
title_vectors.shape

## What We Have So Far
Let's take a moment and summarize we have so far.

We want to predict the *number of views* a video will get, based of it's *title*,*category*, *description*, and *tags* (Since those are the variables we can control when making our video).

We then vectorized our title, description, and tags so that we can use them in our models.

In [ ]:
variables = df[["title","category","description","tags","views"]]
variables.head()

In [ ]:
title_vectors.shape

In [ ]:
tag_vectors.shape

In [ ]:
description_vectors.shape